# Dockerizing a R App

I've heard good things about Docker and I have yet to put a R project in my portfolio, despite claiming knowledge of R programming, so I figured that I'd (figuratively) kill two birds with one stone by containerizing a web app made in R.

## R Plumber

Plumber is a R package for designing web apps. FYI, [here](https://www.rplumber.io/docs/index.html)'s a link to the documentation.

Now, in this project I'll build a web app using the auto.arima function from R's forecast package. FYI, [here](https://cran.r-project.org/web/packages/forecast/forecast.pdf)'s the documentation for that library, along with a [link](https://otexts.com/fpp2/) to the author's excellent book, "Forecasting: Principles and Practice".

Anyways, the web app will call upon a function that uses the auto.arima function to find the best fitting Arima type model for a time series, then return the coefficients of said Arima model.

Here's the R script:

```R
# plumber_auto_arima.R

if(!require(forecast)){
  install.packages("forecast")
}
library(forecast)

#* Automatically fit an ARIMA model
#* @param y the time series
#* @param f:numeric the frequency of the time series
#* @post /auto-arima
function(y, f = 12){
  y <- ts(y, frequency = f)
  model <- auto.arima(y)
  order <- arimaorder(model)
  result <- list(p = order[1],
                 d = order[2],
				 q = order[3],
				 P = order[4],
				 D = order[5],
				 Q = order[6],
				 S = order[7])
  return(result)				 
}

```

Especially important are the comments above the function. The comment @param specifies a parameter for the function, and the @post specifies the request method for the API.

With this script saved, another R program *plumbs* this script and runs the API. That script is below:

```R
if(!require(plumber)){
  install.packages("plumber")
}
library(plumber)

r <- plumb("/home/auto_arima/plumber_auto_arima.R")  
r$run(host='0.0.0.0', port=8000)
```

Since this program will be run inside a Docker container, the filepath of the script is the location of that file inside the Docker image.

Also, to make this run correctly, I am specifying the app to run from the IP Address "0.0.0.0". 

I am not quite sure why port 8000 is used, but I've seen it in plenty of examples, so as the saying goes, monkey see monkey do.

## Docker

To run this app in Docker, you need to first build the Docker image. To build the image, you need a Docker file that specifies the operating system, environment, packages, etc. that you need to run the app.

Luckily, there's an example of a plumber app in [Docker Hub](https://hub.docker.com/). Below is a modified Docker file, ~~stolen~~ borrowed from trestletech/plumber:

```Docker
FROM rocker/r-base
MAINTAINER Jeff Allen <docker@trestletech.com>

RUN apt-get update -qq && apt-get install -y \
  git-core \
  libssl-dev \
  libcurl4-gnutls-dev

## RUN R -e 'install.packages(c("devtools"))'
## RUN R -e 'devtools::install_github("trestletech/plumber")'
RUN install2.r plumber

RUN install2.r forecast

RUN mkdir /home/auto_arima

## RUN R -e "install.packages(c('forecast', 'plumber'), dependencies=TRUE)"
  
COPY plumber_auto_arima.R /home/auto_arima/plumber_auto_arima.R
COPY plumber.R /home/auto_arima/plumber.R

EXPOSE 8000

CMD R -e "source('/home/auto_arima/plumber.R')"
```

The main changes to the trestletech/plumber code are installing the forecast package, making a /home/auto_arima directory, copying my R scripts into that directory, and running the plumber.R script.

Now with the Docker file and R scripts complete, building the image is pretty easy. You simply use the "docker buid" command like so:

```Docker
docker build -t dockerbuild-test "C:\Users\mwtichen\Documents\R" 
```

Note: the -t flag is used to name the image.

To run this in a container, you simply use the "docker run" command, like this:

```Docker
docker run -p 8000:8000  dockerbuild-test
```

Note: the -p flag is mapping port 8000 in the R program to port 8000 on my machine.

## Testing the API

After using the "docker run" command, we have a web app running. 

Now, let's test it by posting a request.

First I'll import the requests library.

In [1]:
import requests

Next I'll make a dictionary of the data I plan to send.

In [2]:
data = {"y" : [822854, 750702, 696562, 995240, 913112, 825685, 736340, 993853, 907254, 811505, 763514],
        "f" : 4}

Lastly, I'll use the post method to request data from the appropriate URL.

In [3]:
url = "http://192.168.99.100:8000/auto-arima"


r = requests.post(url, json=data)

Here's the content that we get back:

In [4]:
r.content

b'{"p":[0],"d":[1],"q":[0],"P":[0],"D":[1],"Q":[0],"S":[4]}'

Success!!!

## Conclusion

I hope you enjoyed following along. If you understood this as you read along, then either: 1) you already know Docker, or 2) you are a far smarter human being than I.

For next steps, I plan on building a Python app with Flask.

As always, thanks for reading.